# Project 1: Trading with Momentum
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.

### Install Packages and RESTART the Kernel

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 880kB 12.7MB/s ta 0:00:01
    100% |████████████████████████████████| 17.0MB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 992kB/s eta 0:00:01
    100% |████████████████████████████████| 41.5MB 688kB/s eta 0:00:01    63% |████████████████████▌           | 26.5MB 60.7MB/s eta 0:00:01
    100% |████████████████████████████████| 50.0MB 590kB/s eta 0:00:01    83% |██████████████████████████▋     | 41.6MB 55.5MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 21.8MB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 35.0MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 35.8MB/s ta 0:00:01
  Installing build dependencies ... done
    100% |████████████████████████████████| 1.2MB 18.5MB/s ta 0:00:01
  Installing build dependencies ... done
    100% |████████████████████████████████| 112kB 34.5MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 30.2MB/s ta 0:00:01
  Inst

Command "/opt/conda/bin/python -m pip install --ignore-installed --no-user --prefix /tmp/pip-build-env-f44q1sp6 --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- "numpy >= 2.0.0; python_version > '3.8'" "oldest-supported-numpy; python_version <= '3.8'" setuptools>=49.5.0 pybind11 cmake" failed with error code 2 in None


In [12]:
!python -m pip install plotly==3.10.0 --no-cache
# Verify
# Should return plotly==3.10.0
import plotly
print(plotly.__version__)

2.0.15


In [19]:
# Verify
# Should return plotly==3.10.0
import plotly
print(plotly.__version__)

2.0.15


### Load Packages

In [9]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
The data we use for most of the projects is end of day data. This contains data for many stocks, but we'll be looking at stocks in the S&P 500. We also made things a little easier to run by narrowing down our range of time period instead of using all of the data.

In [14]:
df = pd.read_csv('../../data/project_1/eod-quotemedia.csv', parse_dates=['date'], index_col=False)

print(df)
print("-----------")

clos = df.reset_index()

print(clos)
print("-----------")

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')

print(close)
print("-----------")

print('Loaded Data')



             date ticker   adj_close
0      2013-07-01      A 29.99418563
1      2013-07-02      A 29.65013670
2      2013-07-03      A 29.70518453
3      2013-07-05      A 30.43456826
4      2013-07-08      A 30.52402098
5      2013-07-09      A 30.68916447
6      2013-07-10      A 31.17771395
7      2013-07-11      A 31.45983407
8      2013-07-12      A 31.48047700
9      2013-07-15      A 31.72819223
10     2013-07-16      A 31.59057266
11     2013-07-17      A 31.38414330
12     2013-07-18      A 31.58369168
13     2013-07-19      A 31.79012104
14     2013-07-22      A 32.20297975
15     2013-07-23      A 31.97590746
16     2013-07-24      A 32.17545584
17     2013-07-25      A 32.10664605
18     2013-07-26      A 31.37726233
19     2013-07-29      A 31.19835688
20     2013-07-30      A 30.86118893
21     2013-07-31      A 30.77861719
22     2013-08-01      A 31.68002538
23     2013-08-02      A 31.91397865
24     2013-08-05      A 31.61121560
25     2013-08-06      A 31.70754930
2

### View Data
Run the cell below to see what the data looks like for `close`.

In [22]:
project_helper.print_dataframe(close)

AttributeError: module 'plotly.graph_objs' has no attribute 'Table'

### Stock Example
Let's see what a single stock looks like from the closing prices. For this example and future display examples in this project, we'll use Apple's stock (AAPL). If we tried to graph all the stocks, it would be too much information.

In [20]:
apple_ticker = 'AAPL'
project_helper.plot_stock(close[apple_ticker], '{} Stock'.format(apple_ticker))

## Resample Adjusted Prices

The trading signal you'll develop in this project does not need to be based on daily prices, for instance, you can use month-end prices to perform trading once a month. To do this, you must first resample the daily adjusted closing prices into monthly buckets, and select the last observation of each month.

Implement the `resample_prices` to resample `close_prices` at the sampling frequency of `freq`.

In [27]:
def resample_prices(close_prices, freq='M'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    # TODO: Implement Function
    
    #print(close_prices)
    #print("---------------")
    #print(freq)
    #print("---------------")
    prices = close_prices.resample(freq).last()
    
    #print(prices)
    
    
    return prices

project_tests.test_resample_prices(resample_prices)

Tests Passed


### View Data
Let's apply this function to `close` and view the results.

In [29]:
monthly_close = resample_prices(close)
print(monthly_close.loc[:, apple_ticker])
print("----------------")
print(close.loc[:, apple_ticker])
print("----------------")
print('{} Stock - Close Vs Monthly Close'.format(apple_ticker))
print("----------------")
project_helper.plot_resampled_prices(
    monthly_close.loc[:, apple_ticker],
    close.loc[:, apple_ticker],
    '{} Stock - Close Vs Monthly Close'.format(apple_ticker))

date
2013-07-31    58.73000866
2013-08-31    63.64994327
2013-09-30    62.28266407
2013-10-31    68.28583759
2013-11-30    73.07037475
2013-12-31    73.72082947
2014-01-31    65.78133976
2014-02-28    69.56208595
2014-03-31    70.95004943
2014-04-30    78.00222579
2014-05-31    84.14255587
2014-06-30    86.47010113
2014-07-31    88.95449982
2014-08-31    95.84930753
2014-09-30    94.21285594
2014-10-31   100.99244110
2014-11-30   111.69411705
2014-12-31   103.66431211
2015-01-31   110.03180655
2015-02-28   121.11705716
2015-03-31   117.31741727
2015-04-30   117.99626112
2015-05-31   123.34294830
2015-06-30   118.74646370
2015-07-31   114.84110860
2015-08-31   107.23801710
2015-09-30   104.56562594
2015-10-31   113.64795179
2015-11-30   112.99053695
2015-12-31   100.53578968
2016-01-31    92.97124993
2016-02-29    92.84754723
2016-03-31   104.65874623
2016-04-30    90.01478000
2016-05-31    96.47778143
2016-06-30    92.36206594
2016-07-31   100.68044865
2016-08-31   103.05832580
2016-09

## Compute Log Returns

Compute log returns ($R_t$) from prices ($P_t$) as your primary momentum indicator:

$$R_t = log_e(P_t) - log_e(P_{t-1})$$

Implement the `compute_log_returns` function below, such that it accepts a dataframe (like one returned by `resample_prices`), and produces a similar dataframe of log returns. Use Numpy's [log function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html) to help you calculate the log returns.

In [35]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    # TODO: Implement Function
    """
    print(np.log(prices))
    print("---------")
    print(prices.shift(1))
    print("---------")
    print(np.log(prices.shift(1)))
    print("---------")
    print(np.log(prices) - np.log(prices.shift(1)))
    """
    
    
    return np.log(prices) - np.log(prices.shift(1))

project_tests.test_compute_log_returns(compute_log_returns)

Tests Passed


### View Data
Using the same data returned from `resample_prices`, we'll generate the log returns.

In [37]:
monthly_close_returns = compute_log_returns(monthly_close)
print(monthly_close_returns.loc[:, apple_ticker])
project_helper.plot_returns(
    monthly_close_returns.loc[:, apple_ticker],
    'Log Returns of {} Stock (Monthly)'.format(apple_ticker))

date
2013-07-31           nan
2013-08-31    0.08044762
2013-09-30   -0.02171531
2013-10-31    0.09201927
2013-11-30    0.06772063
2013-12-31    0.00886237
2014-01-31   -0.11394918
2014-02-28    0.05588347
2014-03-31    0.01975642
2014-04-30    0.09476126
2014-05-31    0.07577509
2014-06-30    0.02728625
2014-07-31    0.02832630
2014-08-31    0.07465224
2014-09-30   -0.01722060
2014-10-31    0.06948903
2014-11-30    0.10071836
2014-12-31   -0.07460613
2015-01-31    0.05961156
2015-02-28    0.09598802
2015-03-31   -0.03187426
2015-04-30    0.00576971
2015-05-31    0.04431573
2015-06-30   -0.03797801
2015-07-31   -0.03344115
2015-08-31   -0.06849869
2015-09-30   -0.02523595
2015-10-31    0.08329065
2015-11-30   -0.00580146
2015-12-31   -0.11679029
2016-01-31   -0.07822348
2016-02-29   -0.00133143
2016-03-31    0.11974615
2016-04-30   -0.15073114
2016-05-31    0.06933886
2016-06-30   -0.04359639
2016-07-31    0.08623527
2016-08-31    0.02334347
2016-09-30    0.06344815
2016-10-31    0.0043

## Shift Returns
Implement the `shift_returns` function to shift the log returns to the previous or future returns in the time series. For example, the parameter `shift_n` is 2 and `returns` is the following:

```
                           Returns
               A         B         C         D
2013-07-08     0.015     0.082     0.096     0.020     ...
2013-07-09     0.037     0.095     0.027     0.063     ...
2013-07-10     0.094     0.001     0.093     0.019     ...
2013-07-11     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...
```

the output of the `shift_returns` function would be:
```
                        Shift Returns
               A         B         C         D
2013-07-08     NaN       NaN       NaN       NaN       ...
2013-07-09     NaN       NaN       NaN       NaN       ...
2013-07-10     0.015     0.082     0.096     0.020     ...
2013-07-11     0.037     0.095     0.027     0.063     ...
...            ...       ...       ...       ...
```
Using the same `returns` data as above, the `shift_returns` function should generate the following with `shift_n` as -2:
```
                        Shift Returns
               A         B         C         D
2013-07-08     0.094     0.001     0.093     0.019     ...
2013-07-09     0.092     0.057     0.069     0.087     ...
...            ...       ...       ...       ...       ...
...            ...       ...       ...       ...       ...
...            NaN       NaN       NaN       NaN       ...
...            NaN       NaN       NaN       NaN       ...
```
_Note: The "..." represents data points we're not showing._

In [38]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """
    # TODO: Implement Function
    
    return returns.shift(shift_n)

project_tests.test_shift_returns(shift_returns)

Tests Passed


### View Data
Let's get the previous month's and next month's returns.

In [39]:
prev_returns = shift_returns(monthly_close_returns, 1)
lookahead_returns = shift_returns(monthly_close_returns, -1)

project_helper.plot_shifted_returns(
    prev_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Previous Returns of {} Stock'.format(apple_ticker))
project_helper.plot_shifted_returns(
    lookahead_returns.loc[:, apple_ticker],
    monthly_close_returns.loc[:, apple_ticker],
    'Lookahead Returns of {} Stock'.format(apple_ticker))

## Generate Trading Signal

A trading signal is a sequence of trading actions, or results that can be used to take trading actions. A common form is to produce a "long" and "short" portfolio of stocks on each date (e.g. end of each month, or whatever frequency you desire to trade at). This signal can be interpreted as rebalancing your portfolio on each of those dates, entering long ("buy") and short ("sell") positions as indicated.

Here's a strategy that we will try:
> For each month-end observation period, rank the stocks by _previous_ returns, from the highest to the lowest. Select the top performing stocks for the long portfolio, and the bottom performing stocks for the short portfolio.

Implement the `get_top_n` function to get the top performing stock for each month. Get the top performing stocks from `prev_returns` by assigning them a value of 1. For all other stocks, give them a value of 0. For example, using the following `prev_returns`:

```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0.015     0.082     0.096     0.020     0.075     0.043     0.074
2013-07-09     0.037     0.095     0.027     0.063     0.024     0.086     0.025
...            ...       ...       ...       ...       ...       ...       ...
```

The function `get_top_n` with `top_n` set to 3 should return the following:
```
                                     Previous Returns
               A         B         C         D         E         F         G
2013-07-08     0         1         1         0         1         0         0
2013-07-09     0         1         0         1         0         1         0
...            ...       ...       ...       ...       ...       ...       ...
```
*Note: You may have to use Panda's [`DataFrame.iterrows`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.iterrows.html) with [`Series.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nlargest.html) in order to implement the function. This is one of those cases where creating a vecorization solution is too difficult.*

In [48]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    # TODO: Implement Function
    
    stocks = pd.DataFrame(0, index=prev_returns.index, columns=prev_returns.columns)
    
    print(stocks)
    print("----------")
    print(prev_returns)
    print("----------")
    print(prev_returns.iterrows())
    print("----------")

    for date, returns in prev_returns.iterrows():
        print("Largest: ", returns.nlargest(top_n))
        print("--")
        top_tickers = returns.nlargest(top_n).index
        print("Top: ", top_tickers)
        print("--")
        stocks.loc[date, top_tickers] = 1
        print("Stocks: ", stocks.loc[date, top_tickers])
        print("--")
        print("")
        print("")
    return stocks

project_tests.test_get_top_n(get_top_n)

            POEM  WHQ  BUY  UKI  KIZX
2008-08-31     0    0    0    0     0
2008-09-30     0    0    0    0     0
2008-10-31     0    0    0    0     0
2008-11-30     0    0    0    0     0
----------
                  POEM         WHQ         BUY         UKI        KIZX
2008-08-31         nan         nan         nan         nan         nan
2008-09-30         nan         nan         nan         nan         nan
2008-10-31  3.13172138  0.72709204  5.76874778  1.77557845  0.04098317
2008-11-30 -3.78816218 -0.67583590 -4.95433863 -1.67093250 -0.24929051
----------
<generator object DataFrame.iterrows at 0x7fa0d0fee360>
----------
Largest:  Series([], Name: 2008-08-31 00:00:00, dtype: float64)
--
Top:  Index([], dtype='object')
--
Stocks:  Series([], Name: 2008-08-31 00:00:00, dtype: int64)
--


Largest:  Series([], Name: 2008-09-30 00:00:00, dtype: float64)
--
Top:  Index([], dtype='object')
--
Stocks:  Series([], Name: 2008-09-30 00:00:00, dtype: int64)
--


Largest:  BUY    5.76874778
PO

### View Data
We want to get the best performing and worst performing stocks. To get the best performing stocks, we'll use the `get_top_n` function. To get the worst performing stocks, we'll also use the `get_top_n` function. However, we pass in `-1*prev_returns` instead of just `prev_returns`. Multiplying by negative one will flip all the positive returns to negative and negative returns to positive. Thus, it will return the worst performing stocks.

In [43]:
top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)
project_helper.print_top(df_long, 'Longed Stocks')
project_helper.print_top(df_short, 'Shorted Stocks')

10 Most Longed Stocks:
INCY, AMD, AVGO, NFX, SWKS, NFLX, ILMN, UAL, NVDA, MU
10 Most Shorted Stocks:
RRC, FCX, CHK, MRO, GPS, WYNN, DVN, FTI, SPLS, TRIP


## Projected Returns
It's now time to check if your trading signal has the potential to become profitable!

We'll start by computing the net returns this portfolio would return. For simplicity, we'll assume every stock gets an equal dollar amount of investment. This makes it easier to compute a portfolio's returns as the simple arithmetic average of the individual stock returns.

Implement the `portfolio_returns` function to compute the expected portfolio returns. Using `df_long` to indicate which stocks to long and `df_short` to indicate which stocks to short, calculate the returns using `lookahead_returns`. To help with calculation, we've provided you with `n_stocks` as the number of stocks we're investing in a single period.

In [52]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    
    df_long_aligned, lookahead_returns_aligned = df_long.align(lookahead_returns, join='inner', axis=0)
    df_short_aligned, _ = df_short.align(lookahead_returns, join='inner', axis=0)
    
    long_returns = (df_long_aligned * lookahead_returns_aligned).sum(axis=1) / n_stocks
    short_returns = (df_short_aligned * lookahead_returns_aligned).sum(axis=1) / n_stocks
    
    short_returns = df_short * lookahead_returns
    short_returns = short_returns.sum(axis=1) / n_stocks
    
    net_returns = (df_long_aligned - df_short_aligned) * lookahead_returns_aligned / n_stocks   
    
    return net_returns

project_tests.test_portfolio_returns(portfolio_returns)

Tests Passed


### View Data
Time to see how the portfolio did.

In [53]:
expected_portfolio_returns = portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)
project_helper.plot_returns(expected_portfolio_returns.T.sum(), 'Portfolio Returns')

## Statistical Tests
### Annualized Rate of Return

In [55]:
expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print(expected_portfolio_returns_by_date)
print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))

date
2013-07-31    0.00000000
2013-08-31    0.00000000
2013-09-30   -0.00236456
2013-10-31   -0.00222744
2013-11-30   -0.00096828
2013-12-31    0.00264429
2014-01-31   -0.00486972
2014-02-28   -0.00761406
2014-03-31   -0.00893045
2014-04-30   -0.00725501
2014-05-31   -0.00938457
2014-06-30   -0.00247228
2014-07-31    0.01026566
2014-08-31    0.01706372
2014-09-30    0.01259415
2014-10-31    0.03362185
2014-11-30    0.01201786
2014-12-31    0.01152313
2015-01-31   -0.01759709
2015-02-28    0.02254455
2015-03-31    0.00792112
2015-04-30    0.02539667
2015-05-31   -0.01203012
2015-06-30    0.03572643
2015-07-31   -0.00304893
2015-08-31    0.03952277
2015-09-30   -0.01182947
2015-10-31    0.01264570
2015-11-30   -0.00222172
2015-12-31   -0.01901051
2016-01-31    0.00987480
2016-02-29   -0.01067894
2016-03-31   -0.02589224
2016-04-30   -0.00242558
2016-05-31   -0.00056459
2016-06-30   -0.00255248
2016-07-31   -0.03299962
2016-08-31    0.00136873
2016-09-30    0.00989578
2016-10-31    0.0074

The annualized rate of return allows you to compare the rate of return from this strategy to other quoted rates of return, which are usually quoted on an annual basis. 

### T-Test
Our null hypothesis ($H_0$) is that the actual mean return from the signal is zero. We'll perform a one-sample, one-sided t-test on the observed mean return, to see if we can reject $H_0$.

We'll need to first compute the t-statistic, and then find its corresponding p-value. The p-value will indicate the probability of observing a t-statistic equally or more extreme than the one we observed if the null hypothesis were true. A small p-value means that the chance of observing the t-statistic we observed under the null hypothesis is small, and thus casts doubt on the null hypothesis. It's good practice to set a desired level of significance or alpha ($\alpha$) _before_ computing the p-value, and then reject the null hypothesis if $p < \alpha$.

For this project, we'll use $\alpha = 0.05$, since it's a common value to use.

Implement the `analyze_alpha` function to perform a t-test on the sample of portfolio returns. We've imported the `scipy.stats` module for you to perform the t-test.

Note: [`scipy.stats.ttest_1samp`](https://docs.scipy.org/doc/scipy-1.0.0/reference/generated/scipy.stats.ttest_1samp.html) performs a two-sided test, so divide the p-value by 2 to get 1-sided p-value

In [59]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    # TODO: Implement Function
    
    print(expected_portfolio_returns_by_date)
    print("-----")
    print("")
    
    t_stat, p_value_two_sided = stats.ttest_1samp(expected_portfolio_returns_by_date, 0)
    
    print(stats.ttest_1samp(expected_portfolio_returns_by_date, 0))
    print("-----")
    print("")
    print(t_stat)
    print("-----")
    print("")
    print (p_value_two_sided)
    print("-----")
    print("")
    
    p_value_one_sided = p_value_two_sided / 2
    
    return t_stat, p_value_one_sided

project_tests.test_analyze_alpha(analyze_alpha)

2008-08-31    0.00000000
2008-09-30    0.00000000
2008-10-31    0.01859903
2008-11-30   -0.41819699
dtype: float64
-----

Ttest_1sampResult(statistic=-0.94076445661840524, pvalue=0.41622819641354447)
-----

-0.940764456618
-----

0.416228196414
-----

Tests Passed


### View Data
Let's see what values we get with our portfolio. After you run this, make sure to answer the question below.

In [60]:
t_value, p_value = analyze_alpha(expected_portfolio_returns_by_date)
print("""
Alpha analysis:
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(t_value, p_value))

date
2013-07-31    0.00000000
2013-08-31    0.00000000
2013-09-30   -0.00236456
2013-10-31   -0.00222744
2013-11-30   -0.00096828
2013-12-31    0.00264429
2014-01-31   -0.00486972
2014-02-28   -0.00761406
2014-03-31   -0.00893045
2014-04-30   -0.00725501
2014-05-31   -0.00938457
2014-06-30   -0.00247228
2014-07-31    0.01026566
2014-08-31    0.01706372
2014-09-30    0.01259415
2014-10-31    0.03362185
2014-11-30    0.01201786
2014-12-31    0.01152313
2015-01-31   -0.01759709
2015-02-28    0.02254455
2015-03-31    0.00792112
2015-04-30    0.02539667
2015-05-31   -0.01203012
2015-06-30    0.03572643
2015-07-31   -0.00304893
2015-08-31    0.03952277
2015-09-30   -0.01182947
2015-10-31    0.01264570
2015-11-30   -0.00222172
2015-12-31   -0.01901051
2016-01-31    0.00987480
2016-02-29   -0.01067894
2016-03-31   -0.02589224
2016-04-30   -0.00242558
2016-05-31   -0.00056459
2016-06-30   -0.00255248
2016-07-31   -0.03299962
2016-08-31    0.00136873
2016-09-30    0.00989578
2016-10-31    0.0074

### Question: What p-value did you observe? And what does that indicate about your signal?

*#TODO: Put Answer In this Cell*

I observed a p-value of 0.073339. This is above the alpha value of 0.05, indicating that there is moderate evidence against the null hypothesis. Therefore, there is a moderate likelihood that our projected strategy will return the projected rate of return.



## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.